In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import logging
import numpy as np
np.random.seed(2023)

logging.basicConfig(
    level=logging.DEBUG,
    format='[%(asctime)s][%(levelname)-5.5s][%(name)-.20s] %(message)s'
)
LOG = logging.getLogger(__name__)

# 1. Load the data
The data are from SISAP 2023 indexing challenge (LAION dataset). There are `100K`, `300K`, and `10M` versions (also `100M`, but that one wasn't tested with LMI). The queries are not included in the data (they are outside of the dataset).

In [4]:
import os
from urllib.request import urlretrieve
from pathlib import Path
import h5py

def download(src, dst):
    if not os.path.exists(dst):
        os.makedirs(Path(dst).parent, exist_ok=True)
        LOG.info('downloading %s -> %s...' % (src, dst))
        urlretrieve(src, dst)

def prepare(kind, size):
    url = "https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge"
    task = {
        "query": f"{url}/public-queries-10k-{kind}.h5",
        "dataset": f"{url}/laion2B-en-{kind}-n={size}.h5",
    }

    for version, url in task.items():
        target_path = os.path.join("data", kind, size, f"{version}.h5")
        download(url, target_path)
        assert os.path.exists(target_path), f"Failed to download {url}"

[2024-03-18 10:58:52,732][DEBUG][h5py._conv] Creating converter from 7 to 5
[2024-03-18 10:58:52,732][DEBUG][h5py._conv] Creating converter from 5 to 7
[2024-03-18 10:58:52,732][DEBUG][h5py._conv] Creating converter from 7 to 5
[2024-03-18 10:58:52,732][DEBUG][h5py._conv] Creating converter from 5 to 7


In [5]:
config = {
    # get the smallest version of the LAION dataset
    'dataset': 'pca32v2',
    'emb': 'pca32',
    'size': '100K',
    # n. of nearest neighbors
    'k': 10,
    # normalize the data to be able to use K-Means
    'preprocess': True
}

In [6]:
# download the data
prepare(config['dataset'], config['size'])

def get_data(data_part, **config):
    return np.array(
        h5py.File(
            os.path.join(
                'data',
                config['dataset'],
                config['size'],
                data_part
            ),
            'r'
        )[config['emb']]
    )

# load the data    
data = get_data("dataset.h5", **config)
queries = get_data("query.h5", **config)
data.shape, queries.shape

((100000, 32), (10000, 32))

## 1.2. Pre-process the data
The default distance metric for LAION dataset is the cosine distance. In order for us to use K-Means for partitioning (which operates only with Euclidean distances), we need to **normalize the data to unit length** (i.e., a single vector will sum up to 1). Data normalized like this can continue to be used with euclidean distance.

In [7]:
# data characteristic before:
sum(data[0])

0.4463985259644687

In [8]:
from sklearn import preprocessing
if config['preprocess']:
    data = preprocessing.normalize(data)
    queries = preprocessing.normalize(queries)

In [9]:
# data characteristics after
sum(data[0])

1.004468702711165

In [10]:
import pandas as pd
# data to pandas
data = pd.DataFrame(data)
# index from one (needed to fit the evaluation procedure later)
data.index += 1

# 2. Build the index

In [11]:
from li.BuildConfiguration import BuildConfiguration
from li.clustering import algorithms
from li.LearnedIndexBuilder import LearnedIndexBuilder

[2024-03-18 10:58:56,559][INFO ][faiss.loader] Loading faiss with AVX2 support.
[2024-03-18 10:58:56,559][INFO ][faiss.loader] Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
[2024-03-18 10:58:56,560][INFO ][faiss.loader] Loading faiss.
[2024-03-18 10:58:56,579][INFO ][faiss.loader] Successfully loaded faiss.


In [12]:
n_categories = [10, 10, 10]

build_config = BuildConfiguration(
    # which clustering algorithm to use
    algorithms['faiss_kmeans'],
    # how many epochs to train for
    200,
    # what model to use (see li/model.py
    'MLP',
    # what learning rate to use
    0.01,
    # how many categories at what level to build LMI for
    # 10, 10 results in 100 buckets in total
    n_categories
)

## 1, 2, 3 levels

In [13]:
first_build_data = data.iloc[:10_000]
builder = LearnedIndexBuilder(first_build_data, build_config)
li, data_prediction, n_buckets_in_index, build_t, cluster_t = builder.build()
data_prediction.shape

[2024-03-18 10:58:59,528][DEBUG][li.LearnedIndexBuild] Training the root model.
[2024-03-18 10:58:59,981][DEBUG][li.LearnedIndexBuild] Epochs: 200, step: 20
[2024-03-18 10:59:03,845][DEBUG][li.LearnedIndexBuild] Epoch 20 | Loss 1.36070
[2024-03-18 10:59:07,599][DEBUG][li.LearnedIndexBuild] Epoch 40 | Loss 0.59687
[2024-03-18 10:59:11,184][DEBUG][li.LearnedIndexBuild] Epoch 60 | Loss 0.50983
[2024-03-18 10:59:15,137][DEBUG][li.LearnedIndexBuild] Epoch 80 | Loss 0.26845
[2024-03-18 10:59:18,882][DEBUG][li.LearnedIndexBuild] Epoch 100 | Loss 0.21265
[2024-03-18 10:59:22,764][DEBUG][li.LearnedIndexBuild] Epoch 120 | Loss 0.17393
[2024-03-18 10:59:27,404][DEBUG][li.LearnedIndexBuild] Epoch 140 | Loss 0.05081
[2024-03-18 10:59:31,285][DEBUG][li.LearnedIndexBuild] Epoch 160 | Loss 0.16542
[2024-03-18 10:59:34,970][DEBUG][li.LearnedIndexBuild] Epoch 180 | Loss 0.29115
[2024-03-18 10:59:38,699][DEBUG][li.LearnedIndexBuild] Trained the model in: 39.17102861404419
[2024-03-18 10:59:38,706][DEBUG]

(10000, 3)

In [14]:
insert_data = data.iloc[10_000:11_000]
data_prediction_2, n_buckets_in_index_2, insert_t = builder.insert(insert_data)
data_prediction_2.shape

[2024-03-18 11:01:26,050][DEBUG][li.LearnedIndexBuild] Predicting the root model.
[2024-03-18 11:01:26,052][DEBUG][li.LearnedIndexBuild] Predicting [10, 10] internal models.
[2024-03-18 11:01:26,053][DEBUG][li.LearnedIndexBuild] Predicting level 1.
  0%|          | 0/10 [00:00<?, ?it/s][2024-03-18 11:01:26,056][DEBUG][li.LearnedIndexBuild] Predicting model on path (0, -1, -1).
[2024-03-18 11:01:26,058][DEBUG][li.LearnedIndexBuild] Predicting model on path (1, -1, -1).
[2024-03-18 11:01:26,060][DEBUG][li.LearnedIndexBuild] Predicting model on path (2, -1, -1).
[2024-03-18 11:01:26,062][DEBUG][li.LearnedIndexBuild] Predicting model on path (3, -1, -1).
[2024-03-18 11:01:26,064][DEBUG][li.LearnedIndexBuild] Predicting model on path (4, -1, -1).
[2024-03-18 11:01:26,067][DEBUG][li.LearnedIndexBuild] Predicting model on path (5, -1, -1).
[2024-03-18 11:01:26,069][DEBUG][li.LearnedIndexBuild] Predicting model on path (6, -1, -1).
[2024-03-18 11:01:26,071][DEBUG][li.LearnedIndexBuild] Predict

(1000, 3)

In [15]:
builder.data.shape

(11000, 32)

In [177]:
data_prediction_all = np.vstack((data_prediction, data_prediction_2))
data_prediction_all.shape

(11000, 3)

In [179]:
dists, nns, measured_time = li.search(
    data_navigation=builder.data,
    queries_navigation=queries[:5],
    data_search=builder.data[[col for col in builder.data.columns if type(col) is int]],
    queries_search=queries[:5],
    data_prediction=data_prediction_all,
    n_categories=n_categories,
    n_buckets=1,
    k=10,
)

[2024-03-18 10:52:01,592][INFO ][li.LearnedIndex.Lear] Precomputed bucket order time: 0.04620647430419922
100%|██████████| 1000/1000 [00:00<00:00, 14595.18it/s]


In [52]:
%%time
builder = LearnedIndexBuilder(first_build_data, build_config)
li, data_prediction, n_buckets_in_index, build_t, cluster_t = builder.build()

100%|██████████| 100/100 [00:51<00:00,  1.95it/s]

CPU times: total: 5min 13s
Wall time: 1min 1s


In [54]:
data_prediction

array([[8, 7, 1],
       [5, 7, 0],
       [2, 9, 7],
       ...,
       [9, 5, 7],
       [7, 1, 0],
       [5, 5, 1]], dtype=int64)

In [55]:
def get_unique_values_counts(arr):
    # Convert the 2D array to a structured array with a view of dtype([('x', int), ('y', int)])
    structured_arr = np.ascontiguousarray(arr).view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[1])))

    # Find unique rows (tuples) and their counts
    unique_structured_arr, counts = np.unique(structured_arr, return_counts=True)
    unique_tuples = unique_structured_arr.view(arr.dtype).reshape(-1, arr.shape[1])

    return unique_tuples, counts

unique_tuples, counts = get_unique_values_counts(data_prediction)
unique_tuples[:4], counts

(array([[0, 0, 0],
        [0, 0, 1],
        [0, 0, 2],
        [0, 0, 3]], dtype=int64),
 array([1, 4, 4, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 4, 5,
        1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 6, 3, 1, 1, 3,
        1, 1, 3, 1, 1, 1, 1, 2, 5, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 6,
        3, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 3, 1, 3, 2, 1, 2, 2, 2, 2, 2,
        1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1,
        2, 3, 1, 1, 1, 1, 3, 1, 3, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 3, 2,
        3, 1, 4, 4, 6, 2, 5, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 4, 2,
        2, 3, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 2, 1,
        2, 1, 1, 3, 1, 4, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3,
        2, 1, 3, 4, 3, 2, 1, 3, 5, 3, 3, 2, 3, 2, 2, 1, 5, 1, 2, 2, 1, 2,
        1, 2, 1, 1, 1, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3,
        3, 3, 2, 3, 4, 2, 1, 1, 2, 1, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 4, 1,
        2, 2, 2, 5, 3

## Doing insert

In [56]:
import time
from itertools import product, takewhile
from logging import DEBUG
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import numpy.typing as npt
import pandas as pd
import torch
import torch.utils.data
from li.BuildConfiguration import BuildConfiguration
from li.clustering import ClusteringAlgorithm
from li.LearnedIndex import LearnedIndex
from li.Logger import Logger
from li.model import LIDataset, ModelParameters, NeuralNetwork, data_X_to_torch
from li.PriorityQueue import EMPTY_VALUE
from li.utils import filter_path_idxs, log_runtime
from tqdm import tqdm

In [74]:
data_to_insert = data.iloc[1000:2000].reset_index(drop=True)
data_to_insert.index += 1

In [75]:
data_to_insert.head(2)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
1,0.540832,0.294648,0.170303,0.300801,-0.334042,0.030603,0.010370,0.064329,-0.099502,0.102232,...,-0.235122,0.021346,-0.032289,0.133272,-0.248649,0.047122,0.119099,-0.107760,0.186530,-0.043702
2,-0.475594,-0.244831,-0.088942,0.291982,0.047102,0.044693,-0.112934,0.098264,0.037613,-0.101401,...,0.041023,-0.023160,-0.241260,0.064872,-0.096614,0.078653,0.024111,0.075076,-0.160184,-0.300525


In [76]:
n_levels = builder.config.n_levels
data_prediction: npt.NDArray[np.int64] = np.full(
    (data_to_insert.shape[0], n_levels), fill_value=EMPTY_VALUE, dtype=np.int64
)

In [77]:
data_prediction[:, 0] = builder.root_model.predict(data_X_to_torch(data_to_insert))

In [78]:
data_prediction

array([[ 8, -1, -1],
       [ 9, -1, -1],
       [ 5, -1, -1],
       ...,
       [ 6, -1, -1],
       [ 1, -1, -1],
       [ 6, -1, -1]], dtype=int64)

In [79]:
# TODO: use as _generate_internal_node_paths
def generate_internal_node_paths(
    level: int, n_levels: int, n_categories: List[int]
) -> List[Tuple]:
    """Generates all possible paths to internal nodes at the given `level`.

    Parameters
    ----------
    level : int
        Desired level of the internal nodes.
    n_levels : int
        Total number of levels in the index.
    n_categories : List[int]
        Number of categories for each level of the index.

    Returns
    -------
    List[Tuple]
        List of all possible paths to internal nodes at the given `level`.
    """
    path_combinations = [range(n_categories[lvl]) for lvl in range(level)]
    padding = [[EMPTY_VALUE]] * (n_levels - level)

    return list(product(*path_combinations, *padding))


In [80]:
for level in range(1, builder.config.n_levels):
    internal_node_paths = generate_internal_node_paths(
        level, builder.config.n_levels, builder.config.n_categories
    )
    builder.logger.info(f"Training level {level}.")

    for path in tqdm(internal_node_paths):
        builder.logger.info(f"Training model on path {path}.")
        data_idxs = filter_path_idxs(data_prediction, path)
        builder.logger.info(f"Using data_idxs: {data_idxs.shape}.")
        # +1 as the data is indexed from 1
        data_prediction_per_path = data_to_insert.loc[data_idxs + 1]

        original_pd_indices = data_prediction_per_path.index.values
        predictions = builder.internal_models[path].predict(data_X_to_torch(data_prediction_per_path))
        builder.logger.info(f"Collected predicions of shape: {predictions.shape}.")
        data_prediction[original_pd_indices - 1, level] = predictions


[2024-03-18 10:10:24,321][INFO ][li.LearnedIndexBuild] Training level 1.
  0%|          | 0/10 [00:00<?, ?it/s][2024-03-18 10:10:24,324][INFO ][li.LearnedIndexBuild] Training model on path (0, -1, -1).
[2024-03-18 10:10:24,324][INFO ][li.LearnedIndexBuild] Using data_idxs: (127,).
[2024-03-18 10:10:24,326][INFO ][li.LearnedIndexBuild] Collected predicions of shape: (127,).
[2024-03-18 10:10:24,327][INFO ][li.LearnedIndexBuild] Training model on path (1, -1, -1).
[2024-03-18 10:10:24,328][INFO ][li.LearnedIndexBuild] Using data_idxs: (101,).
[2024-03-18 10:10:24,328][INFO ][li.LearnedIndexBuild] Collected predicions of shape: (101,).
[2024-03-18 10:10:24,328][INFO ][li.LearnedIndexBuild] Training model on path (2, -1, -1).
[2024-03-18 10:10:24,328][INFO ][li.LearnedIndexBuild] Using data_idxs: (83,).
[2024-03-18 10:10:24,328][INFO ][li.LearnedIndexBuild] Collected predicions of shape: (83,).
[2024-03-18 10:10:24,328][INFO ][li.LearnedIndexBuild] Training model on path (3, -1, -1).
[2024

In [82]:
data_prediction.shape

(1000, 3)

In [20]:
LOG.info(f"Total number of buckets in the index: {n_buckets_in_index}")
LOG.info(f"Cluster time: {cluster_t}")
LOG.info(f"Pure build time: {build_t}")

[2023-10-04 09:58:00,083][INFO ][__main__] Total number of buckets in the index: 100
[2023-10-04 09:58:00,085][INFO ][__main__] Cluster time: 0.9519636631011963
[2023-10-04 09:58:00,086][INFO ][__main__] Pure build time: 352.65329146385193


In [21]:
# n_levels == len(n_categories)
# `data_prediction` (dimensions: n of data points x n_levels)
#   stores assignment of every object to a bucket
n_levels = len(n_categories)
assert data_prediction.shape == ((data.shape[0], len(n_categories)))

In [22]:
data_prediction[:2]

array([[8, 7],
       [3, 2]])

# 3. Load search data
In this specific dataset, we use data that has been subject to dimensionality reduction (hence `pca32v2` in `config['dataset']`). This is nice, because it allows us to train faster (on 32-dim. vectors vs. 768) and have faster navigation during search. However during the sequential search phase on the candidate set, we reach better accuracy when searching with the non-reduced set.

Let's load the full dataset now:

In [23]:
config['dataset'] = 'clip768v2'
config['emb'] = 'emb'

prepare(config['dataset'], config['size'])
data_search = get_data("dataset.h5", **config)
queries_search = get_data("query.h5", **config)

data_search = pd.DataFrame(data_search)
data_search.index += 1

data_search.shape, queries_search.shape

[2023-10-04 09:59:08,370][INFO ][__main__] downloading https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/public-queries-10k-clip768v2.h5 -> data/clip768v2/100K/query.h5...
[2023-10-04 09:59:11,056][INFO ][__main__] downloading https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/laion2B-en-clip768v2-n=100K.h5 -> data/clip768v2/100K/dataset.h5...


((100000, 768), (10000, 768))

# 4. Search in the index

In [42]:
# specify the stop condition
bucket=10
# specify the n. of neighbors
k=10

In [43]:
%%time
dists, nns, measured_time = li.search(
    # the 'navigation' data
    data_navigation=data,
    queries_navigation=queries,
    # the 'sequential filtering' data
    data_search=data_search,
    queries_search=queries_search,
    # mapping of object -> bucket
    data_prediction=data_prediction,
    # n. of categories present in index
    n_categories=n_categories,
    # stop condition for the search
    n_buckets=bucket,
    # number of nearest neighbors we're interested in
    k=k
)

[2023-10-04 10:07:00,406][INFO ][li.LearnedIndex.Lear] Precomputed bucket order time: 0.4756293296813965
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 180.81it/s]

CPU times: user 7.44 s, sys: 55.5 ms, total: 7.49 s
Wall time: 7.75 s


In [44]:
# Time to search (broken down into various search parts)
measured_time

defaultdict(float,
            {'inference': 0.05044746398925781,
             'search_within_buckets': 7.249154806137085,
             'seq_search': 4.535206317901611,
             'sort': 0.0,
             'search': 7.75262188911438})

In [27]:
# matrix of the nearest neighbors (`k` for each query)
nns.shape

(10000, 10)

In [28]:
nns[:2]

array([[79172, 15735, 22337, 74173, 41079, 38159, 69015, 92811, 79896,
        13236],
       [14347, 82848, 79302, 85923,  6016, 67067, 54566, 34591, 11620,
        53783]], dtype=uint32)

In [29]:
# matrix of distances to the closest neighbors (`k` for each query)
dists.shape

(10000, 10)

In [30]:
dists[:2]

array([[0.27291209, 0.30623567, 0.3131932 , 0.32404494, 0.33161247,
        0.33278447, 0.34032881, 0.34535122, 0.35354602, 0.36600691],
       [0.19766825, 0.21139383, 0.22871637, 0.23902297, 0.25272477,
        0.25969118, 0.2700808 , 0.2767331 , 0.27809215, 0.28464031]])

# 5. Evaluate the search performance

In [33]:
def get_groundtruth(size="100K"):
    url = f"https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/laion2B-en-public-gold-standard-v2-{size}.h5"

    out_fn = os.path.join("data", f"groundtruth-{size}.h5")
    download(url, out_fn)
    gt_f = h5py.File(out_fn, "r")
    true_I = np.array(gt_f['knns'])
    gt_f.close()
    return true_I

gt = get_groundtruth(config['size'])

[2023-10-04 10:03:59,376][INFO ][__main__] downloading https://sisap-23-challenge.s3.amazonaws.com/SISAP23-Challenge/laion2B-en-public-gold-standard-v2-100K.h5 -> data/groundtruth-100K.h5...


In [36]:
def get_recall(I, gt, k):
    assert k <= I.shape[1]
    assert len(I) == len(gt)

    n = len(I)
    recall = 0
    for i in range(n):
        recall += len(set(I[i, :k]) & set(gt[i, :k]))
    return recall / (n * k)

In [45]:
recall = get_recall(nns, gt, k)
recall

0.87099